<a href="https://colab.research.google.com/github/prostodema/PRACTIC/blob/main/HW_Dementiev3_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [26]:
from google.colab import drive
drive.mount('/content/drive')

#from google.colab import files
#files.upload()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
ile_path = '/content/drive/My Drive/Python/survey_results_public.csv'

survey_data = pd.read_csv(file_path)

# Display the first few rows of the dataset to understand its structure
survey_data.head()

,ResponseId,MainBranch,Age,Employment,RemoteWork,Check,CodingActivities,EdLevel,LearnCode,LearnCodeOnline,...,JobSatPoints_6,JobSatPoints_7,JobSatPoints_8,JobSatPoints_9,JobSatPoints_10,JobSatPoints_11,SurveyLength,SurveyEase,ConvertedCompYearly,JobSat
0,1,I am a developer by profession,Under 18 years old,"Employed, full-time",Remote,Apples,Hobby,Primary/elementary school,Books / Physical media,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,I am a developer by profession,35-44 years old,"Employed, full-time",Remote,Apples,Hobby;Contribute to open-source projects;Other...,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Books / Physical media;Colleague;On the job tr...,Technical documentation;Blogs;Books;Written Tu...,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
2,3,I am a developer by profession,45-54 years old,"Employed, full-time",Remote,Apples,Hobby;Contribute to open-source projects;Other...,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",Books / Physical media;Colleague;On the job tr...,Technical documentation;Blogs;Books;Written Tu...,...,NaN,NaN,NaN,NaN,NaN,NaN,Appropriate in length,Easy,NaN,NaN
3,4,I am learning to code,18-24 years old,"Student, full-time",NaN,Apples,NaN,Some college/university study without earning ...,"Other online resources (e.g., videos, blogs, f...",Stack Overflow;How-to videos;Interactive tutorial,...,NaN,NaN,NaN,NaN,NaN,NaN,Too long,Easy,NaN,NaN
4,5,I am a developer by profession,18-24 years old,"Student, full-time",NaN,Apples,NaN,"Secondary school (e.g. American high school, G...","Other online resources (e.g., videos, blogs, f...",Technical documentation;Blogs;Written Tutorial...,...,NaN,NaN,NaN,NaN,NaN,NaN,Too short,Easy,NaN,NaN


In [31]:
# 1: How many respondents took the survey?

total_respondents = survey_data['ResponseId'].nunique()
print(f"Total respondents: {total_respondents}")

Total respondents: 65437


In [32]:
# 2: How many respondents answered all the questions?
respondents_answered_all = survey_data.dropna().shape[0]
print(f"Respondents who answered all questions: {respondents_answered_all}")

Respondents who answered all questions: 0


In [33]:
# 3: What are the central tendency measures (mean, median, mode) for respondents' work experience (WorkExp)?
survey_data['YearsCodePro'] = pd.to_numeric(survey_data['YearsCodePro'], errors='coerce')  # Конвертуємо у числовий формат
mean_work_exp = survey_data['YearsCodePro'].mean()
median_work_exp = survey_data['YearsCodePro'].median()
mode_work_exp = survey_data['YearsCodePro'].mode()[0] if not survey_data['YearsCodePro'].mode().empty else None
print(f"Mean work experience: {mean_work_exp} years")
print(f"Median work experience: {median_work_exp} years")
print(f"Mode of work experience: {mode_work_exp} years")

Mean work experience: 10.739651773981603 years
Median work experience: 8.0 years
Mode of work experience: 2.0 years


In [34]:
# 4: How many respondents work remotely?
remote_work_count = survey_data[survey_data['RemoteWork'] == 'Fully remote']['ResponseId'].nunique()
print(f"Respondents working remotely: {remote_work_count}")

Respondents working remotely: 0


In [35]:
# 5: What percentage of respondents program in Python?
python_programmers_count = survey_data[survey_data['LanguageHaveWorkedWith'].str.contains('Python', na=False)]['ResponseId'].nunique()
python_programmers_percentage = (python_programmers_count / total_respondents) * 100
print(f"Percentage of respondents programming in Python: {python_programmers_percentage:.2f}%")


Percentage of respondents programming in Python: 47.06%


In [36]:
# 6: How many respondents learned programming through online courses?
online_courses_count = survey_data[survey_data['LearnCode'].str.contains('Online course', na=False)]['ResponseId'].nunique()
print(f"Respondents who learned programming via online courses: {online_courses_count}")


Respondents who learned programming via online courses: 0


In [37]:
# 7: Among respondents who program in Python, grouped by country, what are the average and median annual compensation (ConvertedCompYearly) in each country?
python_programmers = survey_data[survey_data['LanguageHaveWorkedWith'].str.contains('Python', na=False)]
compensation_by_country = python_programmers.groupby('Country')['ConvertedCompYearly'].agg(['mean', 'median'])
print("Mean and median compensation by country for Python programmers:")
print(compensation_by_country)


Mean and median compensation by country for Python programmers:
                                               mean    median
Country                                                      
Afghanistan                             4543.000000    4768.5
Albania                                56295.000000   56295.0
Algeria                                 9053.285714    6230.0
Andorra                               193331.000000  193331.0
Angola                                     6.000000       6.0
...                                             ...       ...
Venezuela, Bolivarian Republic of...   21500.000000    7100.0
Viet Nam                               14014.562500   10180.0
Yemen                                  10297.333333    5333.0
Zambia                                 28123.666667   22803.0
Zimbabwe                               37500.000000   18000.0

[173 rows x 2 columns]


In [38]:
# 8: What are the education levels of the 5 respondents with the highest compensation?
top5_compensation_education = survey_data.nlargest(5, 'ConvertedCompYearly')[['EdLevel', 'ConvertedCompYearly']]
print("Top 5 respondents by compensation and their education levels:")
print(top5_compensation_education)

Top 5 respondents by compensation and their education levels:
                                              EdLevel  ConvertedCompYearly
15837    Bachelor’s degree (B.A., B.S., B.Eng., etc.)           16256603.0
12723  Professional degree (JD, MD, Ph.D, Ed.D, etc.)           13818022.0
28379  Professional degree (JD, MD, Ph.D, Ed.D, etc.)            9000000.0
17593    Bachelor’s degree (B.A., B.S., B.Eng., etc.)            6340564.0
17672  Professional degree (JD, MD, Ph.D, Ed.D, etc.)            4936778.0


In [39]:
# Bonus 1: In each age category, what percentage of respondents program in Python?
age_groups_python = survey_data.groupby('Age').apply(lambda x: (x['LanguageHaveWorkedWith'].str.contains('Python', na=False).sum() / len(x)) * 100)
print("Percentage of Python programmers by age group:")
print(age_groups_python)

Percentage of Python programmers by age group:
Age
18-24 years old       55.922826
25-34 years old       45.773912
35-44 years old       41.520546
45-54 years old       41.910706
55-64 years old       40.427184
65 years or older     37.564767
Prefer not to say     45.341615
Under 18 years old    64.875389
dtype: float64


In [40]:
# Bonus 2.1: 75 percentile
comp_75_percentile = survey_data['ConvertedCompYearly'].quantile(0.75)

In [43]:
# Bonus 2.2: Respondents in the 75th percentile that has compensation
high_earning_remote_workers = survey_data[(survey_data['ConvertedCompYearly'] >= comp_75_percentile) & (survey_data['RemoteWork'] == 'Fully remote')]

In [42]:
# Group
industry_distribution = high_earning_remote_workers['Industry'].value_counts()
print("Top industries for remote workers in the 75th percentile by compensation:")
print(industry_distribution)

Top industries for remote workers in the 75th percentile by compensation:
Series([], Name: count, dtype: int64)
